<a href="https://colab.research.google.com/github/mattmason36/pyTorch-testing/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
%matplotlib inline

In [2]:
# Convert MNIST image files into tensor of 4 dimensions (# of images, height,
# width, color channel)
transform = transforms.ToTensor()

In [3]:
# train data
train_data = datasets.MNIST(root='/cnn_data', train = True, download = True, transform = transform)

100%|██████████| 9.91M/9.91M [00:01<00:00, 6.83MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 161kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.52MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 6.21MB/s]


In [4]:
# Test Data
test_data = datasets.MNIST(root='/cnn_data', train = False, download = True, transform = transform)

In [5]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: /cnn_data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [6]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: /cnn_data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [7]:
# Create a small batch size for images....lets say 10
train_loader = DataLoader(train_data, batch_size = 10, shuffle = True)
test_loader = DataLoader(test_data, batch_size = 10, shuffle = False)

In [26]:
# Define our CNN Model
# Decribe convolutional layer and what it`s doing (2 convolutional layers)
# This is just an example in the next video we`ll build out the actual model
conv1 = nn.Conv2d(1, 6, 3, 1)
conv2 = nn.Conv2d(6, 16, 3, 1)

In [19]:
# Grab one MNIST record
for i, (X_Train, y_train) in enumerate(train_data):
    break

In [20]:
X_Train.shape

torch.Size([1, 28, 28])

In [21]:
x = X_Train.view(1, 1, 28, 28)

In [22]:
# Perform our first convolution
x = F.relu(conv1(x)) # Recrified Linear Unit for our activation function

In [27]:
# 1 single image, 6 is the filters we asked for, 26x26
x.shape

torch.Size([1, 6, 26, 26])

In [28]:
# pass thru the pooling layer
x= F.max_pool2d(x, 2, 2) # kernel of 2 and stride of 2

In [29]:
x.shape # 26/2 = 13

torch.Size([1, 6, 13, 13])

In [30]:
# Do our second convolutional layer
x = F.relu(conv2(x))

In [31]:
x.shape # Agaion we didnt set padding so we loose the 2 pixels of the outline of the image

torch.Size([1, 16, 11, 11])

In [32]:
# Pooling layer
x = F.max_pool2d(x, 2, 2)

In [33]:
x.shape # 11/2 = 5.5 we have to round down, because we can't invent data to round up

torch.Size([1, 16, 5, 5])

In [35]:
((28-2)/2-2)/2

5.5

In [36]:
# Model class
class ConvolutionalNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 3, 1)
        self.conv2 = nn.Conv2d(6, 16, 3, 1)
        # Fully connected
        self.fc1 = nn.Linear(5*5*16, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, X):
        X = F.relu(self.conv1(X))
        X = F.max_pool2d(X, 2, 2) # 2x2 kernel and stride 2
        # Second pass
        X = F.relu(self.conv2(X))
        X = F.max_pool2d(X, 2, 2) # 2x2 kernel and stride 2
        # Flatten
        X = X.view(-1, 16*5*5) # Negative one ss that we can vary the batch size

        # Fully connected layers
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        X = self.fc3(X)
        return F.log_softmax(X, dim = 1)

In [37]:
# Create an instance of our model
torch.manual_seed(41)
model = ConvolutionalNetwork()
model

ConvolutionalNetwork(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [39]:
# Loss function Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001) # Smaller the learning rate the longer its gonna take to train